In [10]:
from linkedin_scraper import Person,actions,Company
from selenium.webdriver import Chrome
import os,dotenv
from selenium.webdriver.edge.options import Options
dotenv.load_dotenv()



#options = Options()
#options.add_argument("headless")  # Enable headless mode
#options.add_argument("window-size=1920,1080")  # Optional: set window size if needed

driver = Chrome()
actions.login(driver=driver,email=os.getenv("LINKEDIN_USER"),password=os.getenv("LINKEDIN_PASSWORD"))


In [2]:

#rick_fox = Person("https://www.linkedin.com/in/rifox?trk=pub-pbmap",driver=driver)
iggy = Person("https://www.linkedin.com/in/andre-iguodala-65b48ab5",driver=driver,scrape=False,get=False)
#Anirudra = Person("https://in.linkedin.com/in/anirudra-choudhury-109635b1",driver=driver)

In [3]:
import os
from typing import List
from time import sleep
import urllib.parse

from linkedin_scraper.objects import Scraper
from linkedin_scraper import constants as c
from linkedin_scraper.jobs import Job

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [4]:
class Event(Scraper):
    def __init__(
        self,
        linkedin_url=None,
        event_title=None,
        organizer=None,
        organizer_linkedin_url=None,
        location=None,
        event_date=None,
        attendees_count=None,
        description=None,
        driver=None,
        close_on_complete=True,
        speakers = [],
        event_link = None,
        scrape=True,
    ):
        super().__init__()
        self.linkedin_url = linkedin_url
        self.event_title = event_title
        self.driver = driver
        self.organizer:Company = organizer
        self.location = location
        self.attendees_count = attendees_count
        self.description = description
        self.event_date = event_date
        self.speakers:list[Person] = speakers
        self.event_link = event_link

        if scrape:
            self.scrape(close_on_complete)


# Get Event list

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import NoSuchElementException , ElementNotInteractableException,TimeoutException
import time
import datetime
iggy.driver = driver
event: WebElement

events = []
for page in range(10):
    driver.get(f"https://www.linkedin.com/search/results/events/?keywords=Networking june&page={page}")
    try:
        iggy.wait_for_element_to_load(by=By.TAG_NAME, name="main")
        time.sleep(1)
        for atendees in driver.find_elements(By.CLASS_NAME,"entity-result__insights"):

            event = atendees.find_element(By.XPATH,"..")
            atendees = atendees.text.split(" ")[0].strip()
            url = event.find_element(By.TAG_NAME,"a").get_attribute("href")
            events.append(Event(
                linkedin_url=url,
                scrape=False,
                driver=driver,
                attendees_count=atendees
            ))
        break
    except NoSuchElementException :
         break

    
    

In [ ]:
event = atendees.find_element(By.XPATH,"..")
event.find_element(By.TAG_NAME,"a").get_attribute("href")


'https://www.linkedin.com/events/7304807296318992385/'

# Event scraping

In [7]:
import tqdm
event:Event
for event in tqdm.tqdm(events):
    if event.organizer is not None:
        continue
    try:
        driver.get(event.linkedin_url)
    except:
        driver.get(event.linkedin_url)
    iggy.wait_for_element_to_load(by=By.TAG_NAME, name="main")
    time.sleep(1)
    try:
        see_more = driver.find_element(By.CSS_SELECTOR, 'a[role="button"].lt-line-clamp__more')
        parent = see_more.find_element(By.XPATH,"..").find_element(By.XPATH,"..").find_element(By.XPATH,"..")
        see_more.click()

        event.description = parent.text.removesuffix("See less").strip()
    except ElementNotInteractableException:
        pass
    except NoSuchElementException:
        pass
    header = driver.find_element(By.TAG_NAME,"h1")
    event.event_title = header.text.strip()
    header = header.find_element(By.XPATH,"..")
    # Magic numbers go brrrr
    organizer_url = header.find_element(By.TAG_NAME,"a").get_attribute("href")
    if "company" in organizer_url:
        event.organizer = Company(organizer_url,scrape=False,get=False,driver=driver)
    else:
        event.organizer = Person(organizer_url,scrape=False,get=False,driver=driver)
    event.event_date = header.find_element(By.CLASS_NAME,"mr2").text
    try:
        event.location = header.find_element(By.TAG_NAME,"p").text
    except NoSuchElementException:
        event.location=None
    event.event_link= header.find_elements(By.TAG_NAME,"a")[-1].get_attribute("href")

    try:
        event.speakers = []
        for speaker in driver.find_element(By.CLASS_NAME,"events-live-speaker-list__speaker-container").find_elements(By.TAG_NAME,"section"):
            event.speakers.append(Person(speaker.find_element(By.TAG_NAME,"a").get_attribute("href"),
                                driver=driver,
                                get=False,
                                scrape=False))
    except NoSuchElementException:
        pass

 35%|███▌      | 35/100 [08:01<14:53, 13.74s/it]


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=55640): Read timed out. (read timeout=120)

In [8]:
event:Event

driver = Chrome()
actions.login(driver=driver,email=os.getenv("LINKEDIN_USER"),password=os.getenv("LINKEDIN_PASSWORD"))
for event in tqdm.tqdm(events):
    try:
        event.driver = driver
        if event.organizer:
            event.organizer.driver = driver
            driver.get(event.organizer.linkedin_url)
            if isinstance(event.organizer,Company):
                event.organizer.scrape(get_employees=False,close_on_complete=False)
            elif isinstance(event.organizer,Person):
                event.organizer.scrape(close_on_complete=False)
        if len(event.speakers) > 0:
            for speaker in event.speakers:
                driver.get(speaker.linkedin_url)
                speaker.scrape()
    except TimeoutError:
        continue

  0%|          | 0/100 [00:00<?, ?it/s]

<selenium.webdriver.remote.webelement.WebElement (session="a78d5fcc50f82ec5c534370f736fb10b", element="f.0B0A9E794BB74DE8640572FD3F86CCAF.d.F559FD3EF6BE0B04954160DBF5EA3C0D.e.220")>


  1%|          | 1/100 [00:15<25:44, 15.60s/it]

<selenium.webdriver.remote.webelement.WebElement (session="a78d5fcc50f82ec5c534370f736fb10b", element="f.0B0A9E794BB74DE8640572FD3F86CCAF.d.A33698155EFFEBDD29BA3E34F9A3DF8B.e.399")>


  2%|▏         | 2/100 [00:30<24:33, 15.03s/it]

...

you are not logged in!
you are not logged in!


  6%|▌         | 6/100 [01:58<36:34, 23.34s/it]

you are not logged in!
<selenium.webdriver.remote.webelement.WebElement (session="a78d5fcc50f82ec5c534370f736fb10b", element="f.0B0A9E794BB74DE8640572FD3F86CCAF.d.35681F263E7FC7C6FFE4810C988580A2.e.147")>


  7%|▋         | 7/100 [02:29<33:12, 21.42s/it]


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6a7b56f65+78965]
	GetHandleVerifier [0x0x7ff6a7b56fc0+79056]
	(No symbol) [0x0x7ff6a78e9dda]
	(No symbol) [0x0x7ff6a79402ea]
	(No symbol) [0x0x7ff6a794059c]
	(No symbol) [0x0x7ff6a7932c8c]
	(No symbol) [0x0x7ff6a79686af]
	(No symbol) [0x0x7ff6a7932b56]
	(No symbol) [0x0x7ff6a7968880]
	(No symbol) [0x0x7ff6a7990dbe]
	(No symbol) [0x0x7ff6a7968443]
	(No symbol) [0x0x7ff6a7931311]
	(No symbol) [0x0x7ff6a79320a3]
	GetHandleVerifier [0x0x7ff6a7e0e26d+2926461]
	GetHandleVerifier [0x0x7ff6a7e08993+2903715]
	GetHandleVerifier [0x0x7ff6a7e26aed+3026941]
	GetHandleVerifier [0x0x7ff6a7b716fe+187406]
	GetHandleVerifier [0x0x7ff6a7b796ef+220159]
	GetHandleVerifier [0x0x7ff6a7b5faf4+114692]
	GetHandleVerifier [0x0x7ff6a7b5fca9+115129]
	GetHandleVerifier [0x0x7ff6a7b464d8+10728]
	BaseThreadInitThunk [0x0x7ffe13dde8d7+23]
	RtlUserThreadStart [0x0x7ffe14fbc5dc+44]


In [9]:
event:Event
for event in events:
    print(event.attendees_count)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
driver.find_element(By.CLASS_NAME,"inline-block").find_element(By.TAG_NAME,"a").get_attribute("href")